In [ ]:
from __future__ import absolute_import, division, print_function
from collections import defaultdict
import matplotlib.pyplot as plt
from chempy.chemistry import Reaction, Substance, ReactionSystem
from chempy.kinetics.ode import get_odesys
from chempy.util.graph import rsys2graph
import sympy
sympy.init_printing()
%matplotlib inline

In [ ]:
A, B, C, D = map(Substance, 'ABCD')
One = sympy.S.One
reactions = r0, r1, r2 = [
    Reaction({'A'}, {'B'}, 4*One/100, name='R1: A cons.'),
    Reaction({'B', 'C'}, {'A', 'C'}, 10**(4*One), name='R2: A reform.'),
    Reaction({'B': 2}, {'B', 'C'}, 3*10**(7*One), name='R3: C form.')
]
rsys = ReactionSystem(reactions, (A, B, C))
rsys

In [ ]:
from IPython.display import HTML
HTML(rsys.unimolecular_html_table()[0] + rsys.bimolecular_html_table()[0])

In [ ]:
rsys2graph(rsys, 'robertson.png', save='.')
from IPython.display import Image; Image('robertson.png')

In [ ]:
odesys = get_odesys(rsys, include_params=True)[0]
odesys.exprs

In [ ]:
odesys.get_jac()

In [ ]:
c0 = defaultdict(float, {'A': 1})
odesys.integrate(1e10, c0, integrator='cvode', nsteps=2000)
odesys.plot_result()
plt.gca().set_xscale('log'); plt.gca().set_yscale('log'); _ = plt.legend(loc='best')

For larger systems it is easy to loose track of what substances are actually playing a part, here the html table can help:

In [ ]:
rsys.substances['D'] = D
HTML(rsys.unimolecular_html_table()[0] + rsys.bimolecular_html_table()[0])

We could also have parsed the reactions from a string:

In [ ]:
import sympy
rsys2 = ReactionSystem.from_string("""
A -> B; sp.Symbol('k1')
B + C -> A + C; sp.Symbol('k2')
2 B -> B + C; sp.Symbol('k3')
""", rxn_parse_kwargs=dict(globals_={'sp': sympy}), substance_factory=lambda formula: Substance(formula))

In [ ]:
odesys2 = get_odesys(rsys2, include_params=False)[0]
odesys2.exprs, odesys2.params

In [ ]:
odesys2.names, odesys2.param_names

In [ ]:
rsys3 = ReactionSystem.from_string("""
A -> B; MassAction(unique_keys=('k1',))
B + C -> A + C; MassAction(unique_keys=('k2',))
2 B -> B + C; MassAction(unique_keys=('k3',))
""", substance_factory=lambda formula: Substance(formula))

In [ ]:
rsys3

In [ ]:
rsys3.substance_names()

In [ ]:
odesys3, param_keys3, unique_keys3, p_units3 = get_odesys(rsys3, include_params=False)
param_keys3, unique_keys3

In [ ]:
odesys3.exprs, odesys3.params, odesys3.names, odesys3.param_names

In [ ]:
odesys3.integrate(1e9, {'A': 1.0, 'B': 0, 'C': 0}, {'k1': 0.04, 'k2': 1e4, 'k3': 3e7})